# Cleanup Process

There are two main schools of thought when it comes to cleaning data for machine learning projects:

1. clean entire data, then split into train/test sets and proceeds with preprocessing for ML
2. split raw data into train/test sets, setup a reusable cleaning pipeline

The first method is easier for smaller projects that will not scale nor expect future data.

The second is the standard process for an online solution, it scales to deployment in production with new data.

Because of the large number of duplicated Tweets (mostly retweets) and because I'm not implementing an online solution, I'll prefer a hybric approach: remove duplicates prior to splitting into train/test sets, but save any further cleanup for the analytic pipeline.

This notebook is not the implementation of this process, it is an exploration of the data so that this process. See `dedupe.py` for actual deduplication script which is run from the command line: ```python dedupe.py```.

Here I am prematurely creating the variable "Retweet", which is essentially feature enginering (a step down the pipeline), so that I can compare the percentage of retweets in both duplicated Tweets and deduplicated Tweets, gain insight into the amount of duplication and tune the Tweet ingestion schedule. Too many Tweets on a single day isn't worth the small gains in non-duplicated Tweets. I've got ingestion tuned to 25% duplication at 50k Tweets a day so I'm ingesting ~37,500 unique Tweets daily.

In [1]:
import os
import re
import time
import pandas as pd

In [2]:
def load_todays_data():
    filepath = os.path.join("..","data","1_raw","tweets")
    today_prefix = datetime.datetime.now().strftime("%Y%m%d")
    dfm = []
    for f in os.listdir(filepath):
        if re.match(today_prefix, f):
            dfm.append(pd.read_csv(os.path.join(filepath, f)))
    df = pd.concat(dfm)
    df = df.reset_index(drop=True)
    return df

In [3]:
def load_all_data():
    filepath = os.path.join("..","data","1_raw","tweets")
    dfm = []
    for f in os.listdir(filepath):
        dfm.append(pd.read_csv(os.path.join(filepath, f)))
    df = pd.concat(dfm)
    df = df.reset_index(drop=True)
    return df

In [4]:
df = load_all_data()
df.shape

(975400, 5)

In [5]:
# quick glance at top users with multiple tweets
grouped = df[['User', 'ID']].groupby('User').count().sort_values('ID', ascending=False)
grouped[grouped['ID']>1].head()

,ID
User,
pinkyfaye,57
GetVidBot,52
KenanWaters,43
vmindarling,37
DeadPoolzNutz,35


### Retweet Column

Issue: not present in sentiment140 data - do not use for comparing classifiers between projects and tracking data drift.

Note: this is a feature engineering step that is only present here for early analysis not as a cleanup step.

In [6]:
def is_retweet(col):

    for i in range(len(col)):
        if re.match(r'^RT', col) is not None:
            return 1
        else:
            return 0      
        
def map_is_retweet(col):
   
    bool_map = map(lambda x: is_retweet(x), col)      
    return(list(bool_map)) 

In [7]:
df['Retweet'] = map_is_retweet(df['Text'].values)

In [8]:
df.head()

,ID,Timestamp,User,Text,Polarity,Retweet
0,1302406168791470081,2020-09-06 00:40:02,asarinanamis,RT @Ayshiun: Totally inspired by@/kianamaiart'...,-1,1
1,1302406168766078976,2020-09-06 00:40:02,xleave_thecity,@thiinkinaboutit thank you 🥺,-1,0
2,1302406168170696706,2020-09-06 00:40:02,MsTam_Tam,some of yall retweets really be having me look...,-1,0
3,1302406168120365056,2020-09-06 00:40:02,_thebdawkk,@jeenbeen__ Thank you Jen 🥺 just miss the old ...,-1,0
4,1302406167956602880,2020-09-06 00:40:02,4ranghae1015,"@SJofficial My favorite part is the BAD boy, g...",-1,0


In [9]:
df.tail()

,ID,Timestamp,User,Text,Polarity,Retweet
975395,1312909692356575232,2020-10-05 00:17:17,min9yufluffy,RT @wonubliss: jeonghan's gift to seungcheol:\...,1,1
975396,1312909692285472768,2020-10-05 00:17:17,shawny_strolls,@acciopage394 So like when I’m at work and try...,1,0
975397,1312909692268687360,2020-10-05 00:17:17,melanin_sugar,@yadastarot Maria 🥰 thanx,1,0
975398,1312909692268548097,2020-10-05 00:17:17,atomicoffin,@pamelarenfree i’ve been having crazy back pai...,1,0
975399,1312909692226555904,2020-10-05 00:17:17,Dory24960234,Bless morning♥️thankyou Lord 🥰♥️,1,0


## POC for Deduplicating on Text

In [10]:
data = {'Text': ['apple','orange','banana','apple','banana','banana','mango']
       , 'ID': [234309, 349102, 930443, 898344, 229945, 690346, 893427]
       , 'Retweet': [0, 0, 0, 1, 1, 0, 1]}
test = pd.DataFrame(data, columns = ['Text','ID','Retweet'])
test

,Text,ID,Retweet
0,apple,234309,0
1,orange,349102,0
2,banana,930443,0
3,apple,898344,1
4,banana,229945,1
5,banana,690346,0
6,mango,893427,1


In [11]:
dupes = test[test['Text'].duplicated(keep='first')]
dupes

,Text,ID,Retweet
3,apple,898344,1
4,banana,229945,1
5,banana,690346,0


In [12]:
dupes.shape[0]/test.shape[0] # % dupes

0.42857142857142855

In [13]:
dupes[dupes['Retweet']==1].shape[0]/dupes.shape[0] # % retweets in dupes

0.6666666666666666

In [14]:
dupes['ID'] # dup ids, remove from test

3    898344
4    229945
5    690346
Name: ID, dtype: int64

In [15]:
deduped_test = test[~test.ID.isin(dupes['ID'])]
deduped_test

,Text,ID,Retweet
0,apple,234309,0
1,orange,349102,0
2,banana,930443,0
6,mango,893427,1


In [16]:
deduped_test[deduped_test['Retweet']==1].shape[0]/deduped_test.shape[0] # % retweets in deduped data 

0.25

Then check for polarity and class balance.

## Deduplicating Data

In [17]:
dupes = df[df['Text'].duplicated(keep='first')]

In [22]:
print("% dupes: " + str(100*round(dupes.shape[0]/df.shape[0], 4)))

% dupes: 25.28


In [23]:
print("% retweets in dupes: " + str(100*round(dupes[dupes['Retweet']==1].shape[0]/dupes.shape[0], 4))) 

% retweets in dupes: 99.81


In [24]:
deduped_df = df[~df.ID.isin(dupes['ID'])]

In [25]:
deduped_df.shape

(728838, 6)

In [26]:
print("% retweets in deduped data: " + \
      str(100*round(deduped_df[deduped_df['Retweet']==1].shape[0]/deduped_df.shape[0], 4)))

% retweets in deduped data: 30.5


In [28]:
df[['ID','Polarity']].groupby('Polarity').count() # original polarity class balance

,ID
Polarity,
-1,487700
1,487700


In [42]:
polarity_df = deduped_df[['ID','Polarity']].groupby('Polarity').count() # polarity class balance after deduping
pct_diff = 100*round(abs(polarity_df['ID'][1] - polarity_df['ID'][-1]) / sum(polarity_df['ID']), 4)
print("% diff in target (Polarity): " + str(pct_diff))

% diff in target (Polarity): 5.82


In [30]:
deduped_df.index = range(len(deduped_df.index))

In [31]:
deduped_df.head()

,ID,Timestamp,User,Text,Polarity,Retweet
0,1302406168791470081,2020-09-06 00:40:02,asarinanamis,RT @Ayshiun: Totally inspired by@/kianamaiart'...,-1,1
1,1302406168766078976,2020-09-06 00:40:02,xleave_thecity,@thiinkinaboutit thank you 🥺,-1,0
2,1302406168170696706,2020-09-06 00:40:02,MsTam_Tam,some of yall retweets really be having me look...,-1,0
3,1302406168120365056,2020-09-06 00:40:02,_thebdawkk,@jeenbeen__ Thank you Jen 🥺 just miss the old ...,-1,0
4,1302406167956602880,2020-09-06 00:40:02,4ranghae1015,"@SJofficial My favorite part is the BAD boy, g...",-1,0


In [32]:
deduped_df.tail()

,ID,Timestamp,User,Text,Polarity,Retweet
728833,1312909692356575232,2020-10-05 00:17:17,min9yufluffy,RT @wonubliss: jeonghan's gift to seungcheol:\...,1,1
728834,1312909692285472768,2020-10-05 00:17:17,shawny_strolls,@acciopage394 So like when I’m at work and try...,1,0
728835,1312909692268687360,2020-10-05 00:17:17,melanin_sugar,@yadastarot Maria 🥰 thanx,1,0
728836,1312909692268548097,2020-10-05 00:17:17,atomicoffin,@pamelarenfree i’ve been having crazy back pai...,1,0
728837,1312909692226555904,2020-10-05 00:17:17,Dory24960234,Bless morning♥️thankyou Lord 🥰♥️,1,0


---